tensorflow==2.1

keras==2.3.1

In [ ]:
pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
!pip install q tensorflow==2.1
!pip install q keras==2.3.1

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
import efficientnet.keras as efn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras import backend as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation 
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.engine.topology import Layer

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
class Generator_xandy(object):
    def __init__(self):
        train_dir =  "/content/drive/My Drive/eff"
        test_dir = "/content/drive/My Drive/eff_test"
        batch_size=20
        datagen = ImageDataGenerator(zoom_range=[0.95, 1.0],width_shift_range =0.1,height_shift_range=0.1, brightness_range=[0.95, 1.05], rescale=1/255.)

        train_gen = datagen.flow_from_directory(
            train_dir,
            target_size=(224,224),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True,
            subset = "training" 
        )

        self.gene = train_gen

    def __iter__(self):

        return self

    def __next__(self): 
        X, Y = self.gene.next()
        #元はreturn X, Y
        return [X,Y], Y

train_generator=Generator_xandy()
val_generator=Generator_xandy()

Found 250 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


In [15]:
#arcface layerの実装
class Arcfacelayer(Layer):
    def __init__(self, output_dim, s=30, m=0.50, easy_magin=False):
        self.output_dim = output_dim
        self.s = s
        self.m = m
        self.easy_magin = easy_magin
        super(Arcfacelayer, self).__init__()

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[0][1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Arcfacelayer, self).build(input_shape)

    def call(self, x):
        y = x[1]
        x_normalize = tf.math.l2_normalize(x[0]) #|x = x'/ ||x'||2
        k_normalize = tf.math.l2_normalize(self.kernel) # Wj = Wj' / ||Wj'||2

        cos_m = K.cos(self.m)
        sin_m = K.sin(self.m)
        th = K.cos(np.pi - self.m)
        mm = K.sin(np.pi - self.m) * self.m

        cosine = K.dot(x_normalize, k_normalize) # W.Txの内積

        sine = K.sqrt(1.0 - K.square(cosine))
        phi = cosine * cos_m - sine * sin_m

        if self.easy_magin:
            phi = tf.where(cosine > 0, phi, cosine) 

        else:
            phi = tf.where(cosine > th, phi, cosine - mm) 

        output = (y * phi) + ((1.0 - y) * cosine) # true cos(θ+m), False cos(θ)
        output *= self.s

        return output

    def compute_output_shape(self, input_shape):

        return (input_shape[0][0], self.output_dim)


In [16]:
K.clear_session()
n_categories=5
#B3の部分をB0~B7と変えるだけでモデルを変更可能
base_model = efn.EfficientNetB3(input_shape=(224,224,3), weights="imagenet",include_top=False)

#ArcFaceLayerをくっつける
x = base_model.output
yinput = Input(shape=(n_categories,))
hidden = GlobalAveragePooling2D()(x)
x = Arcfacelayer(5, 40, 0.05)([hidden,yinput])
prediction = Activation('softmax')(x)
efn_model = Model(inputs=[base_model.input,yinput],outputs=prediction)

efn_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

efn_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 40) 1080        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 40) 160         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 40) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [17]:
from keras import callbacks


callbacks_list = [
                  callbacks.ReduceLROnPlateau(
                      monitor="val_loss",
                      factor=0.8,
                      patience=5,
                      verbose=1)]


#学習
efn_model.fit_generator(train_generator, steps_per_epoch=5, epochs=300, verbose=1,callbacks=callbacks_list, 
                         validation_data=val_generator, validation_steps=5, class_weight=None, max_queue_size=10, workers=1, 
                        use_multiprocessing=False, 
                        shuffle=True, initial_epoch=0)

Epoch 1/300
5/5 [==============================] - 44s 9s/step - loss: 2.6875 - accuracy: 0.0400 - val_loss: 2.7256 - val_accuracy: 0.0000e+00
Epoch 2/300
5/5 [==============================] - 6s 1s/step - loss: 1.2309 - accuracy: 0.6600 - val_loss: 1.5989 - val_accuracy: 0.3300
Epoch 3/300
5/5 [==============================] - 6s 1s/step - loss: 0.7894 - accuracy: 0.8556 - val_loss: 2.4813 - val_accuracy: 0.6222
Epoch 4/300
5/5 [==============================] - 6s 1s/step - loss: 0.6418 - accuracy: 0.9700 - val_loss: 1.3333 - val_accuracy: 0.7300
Epoch 5/300
5/5 [==============================] - 6s 1s/step - loss: 0.5055 - accuracy: 0.9800 - val_loss: 0.9085 - val_accuracy: 0.6600
Epoch 6/300
5/5 [==============================] - 5s 1s/step - loss: 0.3654 - accuracy: 1.0000 - val_loss: 0.8971 - val_accuracy: 0.6667
Epoch 7/300
5/5 [==============================] - 6s 1s/step - loss: 0.3776 - accuracy: 0.9900 - val_loss: 0.8518 - val_accuracy: 0.7300
Epoch 8/300
5/5 [============

In [19]:
def get_hold_vector(model, classes=[0,1,2,3,4]):
    """
    classes: クラス名のリスト　イメージの名前はこのリスト名にしてください
    hold_dir: str イメージが入ったフォルダpath
    """

    hold_vector = np.zeros((5,1536))
    train_path="/content/drive/My Drive/eff"

    for clas in classes:
        img_array = np.empty((0, 224,224,3))
        for i in range(50):
            imagepath=glob.glob("/content/drive/My Drive/eff/{}/*".format(clas))[i]

            img = tf.keras.preprocessing.image.load_img(imagepath, target_size=(224,224))
            array = tf.keras.preprocessing.image.img_to_array(img).reshape(1, 224, 224, 3)
            img_array = np.vstack((img_array, array))


        img_array = img_array/255.0
        vector = model.predict(img_array)
        vector = vector.mean(axis=0)
        hold_vector[clas-1, :]=vector
    return hold_vector
#予測用のモデル
predict_model=Model(efn_model.get_layer(index=0).input, efn_model.get_layer(index=-4).output)
predict_model.save("/content/drive/My Drive/eff/pro_eff3_model.h5")

#学習データ（5種類のペットボトル）のベクトル
hold_vector=get_hold_vector(model=predict_model)
np.save(
    "/content/drive/My Drive/eff/pro_hold_vector.npy", # データを保存するファイル名
    hold_vector,  # 配列型オブジェクト（listやnp.array)
)
